In [1]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path+"\\Source")

from Source.Market import Market, MockStockGeometricBrownianMotion
from Source.Agent import Agent


In [2]:
initial_cash = 10000
asset_test = {'Cash': initial_cash, 'Stock': 0}
agent_test = Agent('agent_test', asset_test)

stock_move_std = 0.01  # stock has daily volatility 1%
observation_std = 0.01  # stock has observation error 1%

# stock follow GBM with 0 drift and const daily volatility
market_test = Market([MockStockGeometricBrownianMotion('Stock', 100, 0, stock_move_std)])
market_test.mark_current_value_to_record(0)

current_stock_value = market_test.check_value('Stock')
predicted_value = market_test.check_prediction('Stock', observation_std)

# Example Strategy
agent_test._trading_intention = {'Stock': 10} if predicted_value > current_stock_value else {'Stock': -10}

agent_test.trade(market_test, 0)
agent_test.mark_holding_values(market_test, 0)

for time in range(1, 252):  # 252 business days in a year
    market_test.evolve(time)
    market_test.mark_current_value_to_record(time)

    current_stock_value = market_test.check_value('Stock')
    predicted_value = market_test.check_prediction('Stock', observation_std)
    
    # Example Strategy
    agent_test._trading_intention = {'Stock': 10} if predicted_value > current_stock_value else {'Stock': -10}
    
    agent_test.trade(market_test, time)
    agent_test.mark_holding_values(market_test, time)

print(f"Average Return: {agent_test.calculate_average_return(): .6f}")
print(f"Return Std: {agent_test.calculate_std_return(): .6f}")
print(f"Strategy Sharpe Ratio: {agent_test.calculate_sharpe_ratio(): .6f}")
print(f"Max Drawdown: {agent_test.calculate_max_drawdown() / initial_cash: .6f}")


Average Return:  0.000713
Return Std:  0.006978
Strategy Sharpe Ratio:  0.102151
Max Drawdown:  0.067331


Objective: Maximize Average Return

Constraints: 
	1. prob(Return Std > Return_Std_Limit) < small_number
	2. prob(Max Drawdown > Max_Drawdown_Limit) < small_number

Example: Return_Std_Limit = 0.2, Max_Drawdown_Limit = 0.1, small_number = 0.05

Research Proposal
1. Constant observation_std, No Transaction Cost
2. Random observation_std, observation_std follows a Gaussian Distribution but the agent knows the observation_std
3. Transaction Cost